In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data= pd.read_csv("seattle-weather.csv")
data.dropna(inplace=True)
data

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
...,...,...,...,...,...,...
1456,2015-12-27,8.6,4.4,1.7,2.9,rain
1457,2015-12-28,1.5,5.0,1.7,1.3,rain
1458,2015-12-29,0.0,7.2,0.6,2.6,fog
1459,2015-12-30,0.0,5.6,-1.0,3.4,sun


In [ ]:
data.duplicated().sum()

0

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
import plotly.express as px
fig = px.line(data, x='date', y=['temp_max', 'temp_min'],
              labels={'value': 'Temperature (°C)', 'date': 'Date'},
              title='Daily Max and Min Temperatures (2012-2015)')

fig.for_each_trace(lambda t: t.update(name={'temp_max': 'Max Temp', 'temp_min': 'Min Temp'}[t.name],
                                          line=dict(color='#636efb' if t.name == 'temp_max' else '#01008c')))
fig.update_layout(
    legend_title_text='Temperature Type',
    xaxis_title='Date',
    yaxis_title='Temperature (°C)',
    template='plotly_white'
)

fig.show()

In [ ]:
weather_counts = data['weather'].value_counts().reset_index()
weather_counts.columns = ['Weather Type', 'Count']
fig = px.bar(weather_counts, x='Weather Type', y='Count',
             title='Distribution of Weather Types',
             labels={'Weather Type': 'Weather Type', 'Count': 'Count'},
             color='Count',
             color_discrete_map={
                 'drizzle': '#0066cc',
                 'rain': '#004080',
                 'fog': '#003366',
                 'sun': '#005bb5',
             })
fig.show()

In [ ]:
fig = px.scatter(data, x='temp_max', y='precipitation',
                 title='sound',
                 labels={'temp_max': 'beats', 'precipitation': 'frequency'})

fig.update_layout(width=800, height=500)
fig.show()

In [ ]:
fig = px.scatter_matrix(data,
                        dimensions=['precipitation', 'temp_max', 'temp_min', 'wind'],
                        color='weather',
                        title='Pairplot of Weather Data by Weather Type',
                        labels={'weather': 'Weather Type'},
                        height=1000, width=1000)

fig.update_traces()
fig.show()

In [ ]:
training = data.iloc[:,2:3].values

In [ ]:
def df_to_XY(df,window_size=10):
 X_train=[]
 y_train=[]

 for i in range(10,len(training)):
    X_train.append(training[i-10:i,0])
    y_train.append(training[i,0])

 X_train, y_train = np.array(X_train), np.array(y_train)
 return X_train, y_train

In [ ]:
WINDOW = 10
X,y = df_to_XY(data,WINDOW)
print(len(X),len(y))
X_train = X[:800]
y_train = y[:800]
X_val = X[800:1000]
y_val = y[800:1000]
X_test = X[1000:]
y_test = y[1000:]

1451 1451


In [ ]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_val = np.reshape(X_val,(X_val.shape[0],X_val.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
model = Sequential()
model.add(LSTM(units=50, return_sequences = True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
history=model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=100, batch_size=32)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - loss: 269.0073 - val_loss: 272.5475
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 113.3217 - val_loss: 177.1923
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 76.3569 - val_loss: 144.9498
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 72.2200 - val_loss: 124.9456
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 56.7618 - val_loss: 110.4058
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 56.7447 - val_loss: 99.2753
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 54.2286 - val_loss: 91.6858
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 51.1930 - val_loss: 85.7259
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 53.8401 - val_loss: 82.0014
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 55.2824 - val_loss: 79.6459
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 51.8701 - val_loss: 78.6664
Epoch 12/100
25/25 ━━━

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
history_loss = hist[['loss', 'val_loss']].reset_index()

fig = px.line(history_loss,
              x=history_loss.index,
              y=['loss', 'val_loss'],
              labels={'index': 'Epochs', 'value': 'Loss'},
              title='Loss & Val Loss')
fig.update_layout(width=700, height=500)

fig.show()

In [ ]:
train_pred = model.predict(X_train).flatten()
val_pred = model.predict(X_val).flatten()
test_pred = model.predict(X_test).flatten()

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [ ]:
pred = np.concatenate([train_pred, val_pred, test_pred])
df_pred = pd.DataFrame(data["temp_max"].copy())
df_pred.columns = ["actual"]
df_pred = df_pred[WINDOW:]
df_pred["predicted"] = pred

df_long = df_pred[800:].reset_index()
df_long = df_long.melt(id_vars='index', value_vars=['actual', 'predicted'],
                        var_name='Type', value_name='Temperature')

fig_val = px.line(df_long, x='index', y='Temperature', color='Type',
                  title="Validation Results",
                  labels={'Temperature': 'Temperature', 'index': 'Time'},
                  color_discrete_map={'actual': 'blue', 'predicted': 'red'})
fig.update_layout(width=700, height=500)

fig_val.show()

In [ ]:
fig_test = px.line(df_pred[1000:], title="Test Results",
                   labels={'value': 'Temperature', 'index': 'Time'},
                   line_shape='linear')
fig_test.update_traces(name='Actual', line_color='blue')
fig_test.add_scatter(x=df_pred.index[1000:], y=df_pred['predicted'][1000:],
                      mode='lines', name='Predicted', line=dict(color='red'))
fig.update_layout(width=700, height=500)
fig_test.show()